In [ ]:
!pip install -Uq openai-agents

: 

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from pydantic import BaseModel
from agents import (
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig)
from google.colab import userdata

In [5]:
gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

In [6]:
from agents import set_default_openai_client, set_tracing_disabled
set_default_openai_client(external_client)
set_tracing_disabled(True)

In [14]:
import asyncio
from dataclasses import dataclass

from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class UserInfo1:
    name: str
    uid: int
    location: str = "UMT"

@function_tool
async def fetch_user_age(wrapper: RunContextWrapper[UserInfo1]) -> str:
    '''Returns the age of the user.'''
    return f"User {wrapper.context.name} is 89 years old"

@function_tool
async def fetch_user_location(wrapper: RunContextWrapper[UserInfo1]) -> str:
    '''Returns the location of the user.'''
    return f"User {wrapper.context.name} is from {wrapper.context.location}"

async def main():
    user_info = UserInfo1(name="ABCDEF", uid=123)

    agent = Agent[UserInfo1](
        name="Assistant",
        tools=[fetch_user_age],
        model=model
    )

    result = await Runner.run(
        starting_agent=agent,
        input="What is the age of the user? current location of his/her?",
        context=user_info,
    )

    print(result.final_output)
    # The user John is 47 years old.

if __name__ == "__main__":
    asyncio.run(main())

The user is 89 years old. I do not have the ability to determine their current location.



### LLM/cotext


In [ ]:
import asyncio
from dataclasses import dataclass

from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class UserInfo:
    name: str
    uid: int

@dataclass
class Skill:
  skills: list[str]

@function_tool
async def greet_user(context: RunContextWrapper, greeting: str) -> str:
  """Greets the User with their name.
  Args:
    greeting: A specialed greeting message for user
  """
  name = context.context[0].name
  skills = context.context[1].skills
  return f"Hello {name}, {greeting}, {skills}"

async def main():
    user_info = UserInfo(name="ABCDEF", uid=123)
    skills = Skill(skills=["Python", "Java"])
    agent = Agent(
        name="Assistant",
        tools=[greet_user],
        model=model,
        # Dynamic function context pass in instructions/system prompmt/developer prompt
        instructions="Always greet the user using <function_call>greet_user</function_call> and welcome them to Panaversity"
    )

    result = await Runner.run(
        starting_agent=agent,
        input="Hello what are the user skills and their name",
        context=[user_info,skills]
    )

    print(result.final_output)

asyncio.run(main())

Hello Muhammad Qasim, Welcome to Panaversity! Your skills are Python and Java.



#### Handoff with context